<a href="https://colab.research.google.com/github/DianaBravoPerez/EDP-1/blob/main/jacobina_ej3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Técnica iterativa de Jacobi


**Se califica:**
a) Comentarios y documentación.
b) Que funcione correctamente.
c) Impresión de resultados en una **tabla ordenada**.


In [2]:
import numpy as np
import pandas as pd

def jacobi(A, b, x0=None, tol=1e-8, max_iter=500):
    """
    Implementación del método de Jacobi

    Parámetros
    ----------
    A : array_like (n×n)
        Matriz del sistema (se asume diagonalmente dominante o SPD para convergencia).
    b : array_like (n,)
        Vector del lado derecho.
    x0 : array_like (n,), opcional
        Aproximación inicial x^(0). Si no se da, se usa el vector cero.
    tol : float
        Tolerancia para el criterio de paro, ||x^(k) − x^(k−1)||_∞ < tol.
    max_iter : int
        Máximo número de iteraciones permitidas.

    Retorna
    -------
    x : ndarray (n,)
        Aproximación final.
    k : int
        Iteraciones realizadas.
    tabla : pandas.DataFrame
        Tabla ordenada con k, componentes de x^(k) y el error ||·||_∞ por iteración.
    convergio : bool
        True si el criterio de paro se cumplió; False si se alcanzó max_iter.
    """
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = A.shape[0]
    x_prev = np.zeros(n) if x0 is None else np.array(x0, dtype=float).copy()

    D = np.diag(np.diag(A))           # Parte diagonal
    R = A - D                         # R = A − D
    Dinv = np.diag(1.0/np.diag(D))    # D^{-1}

    registros = []
    convergio = False
    for k in range(1, max_iter+1):
        # Paso 3 (fórmula de Jacobi): x^(k) = D^{-1}(b − R x^(k−1))
        x = Dinv @ (b - R @ x_prev)
        # Paso 4: criterio de paro
        err = np.linalg.norm(x - x_prev, ord=np.inf)
        registros.append([k, *x.tolist(), err])
        if err < tol:
            convergio = True
            break
        # Pasos 5-6: avanzar iteración
        x_prev = x
    cols = ["k"] + [f"x{i+1}" for i in range(A.shape[0])] + ["error_inf"]
    tabla = pd.DataFrame(registros, columns=cols)
    return x, k, tabla, convergio

def arma_sistema_laplace_cuadrado(xs, ys, u_left, u_right, u_bottom, u_top):
    """
    Ensambla el sistema lineal A x = b asociado a ∇^2 u = 0 en una malla rectangular
    con condiciones de Dirichlet dadas en el borde. Se usa el esquema de 5 puntos.

    xs, ys : arrays 1D con los nodos incluyendo fronteras (x0,...,xM), (y0,...,yN)
    u_left(y), u_right(y), u_bottom(x), u_top(x) : funciones de frontera.

    Devuelve A (n×n), b (n,), y una función idx(i,j) que mapea (i,j) interiores a índice 0..n-1.
    """
    xs = np.asarray(xs, float); ys = np.asarray(ys, float)
    Nx = len(xs) - 2  # nodos interiores en x
    Ny = len(ys) - 2  # nodos interiores en y
    n = Nx * Ny
    A = np.zeros((n, n), float)
    b = np.zeros(n, float)
    def idx(i, j):
        # i=1..Nx (x); j=1..Ny (y, de abajo hacia arriba)
        return (j-1) * Nx + (i-1)
    # Ensamble: 4 u_ij − u_{i±1,j} − u_{i,j±1} = (contribuciones de frontera)
    for j in range(1, Ny+1):
        for i in range(1, Nx+1):
            k = idx(i, j)
            A[k, k] = 4.0
            # vecino izquierdo
            if i-1 >= 1: A[k, idx(i-1, j)] = -1.0
            else:        b[k] += u_left(ys[j])
            # vecino derecho
            if i+1 <= Nx: A[k, idx(i+1, j)] = -1.0
            else:         b[k] += u_right(ys[j])
            # vecino abajo
            if j-1 >= 1: A[k, idx(i, j-1)] = -1.0
            else:        b[k] += u_bottom(xs[i])
            # vecino arriba
            if j+1 <= Ny: A[k, idx(i, j+1)] = -1.0
            else:         b[k] += u_top(xs[i])
    return A, b, idx, Nx, Ny


## Haciendo la prueba con el ejercicio visto en clase.
Dominio $[0,2]\times[0,2]$, tamaño de malla $h=2/3$ ⇒ nodos: $0,\;2/3,\;4/3,\;2$.

Condiciones de frontera:
$$\begin{aligned}
u(0,y)&=0, & u(2,y)&=y(2-y),\\
u(x,0)&=0, & u(x,2)&=\begin{cases}x,&0<x<1,\\ 2-x,&1\le x<2.\end{cases}
\end{aligned}$$

**Al ejecutar el código, el usuario elige la tolerancia** y (opcional) el número máximo de iteraciones.

In [3]:
# Nodos incluyendo frontera (h = 2/3)
xs = np.array([0.0, 2/3, 4/3, 2.0])
ys = np.array([0.0, 2/3, 4/3, 2.0])

# Fronteras como funciones
u_left   = lambda y: 0.0
u_right  = lambda y: y*(2-y)
u_bottom = lambda x: 0.0
def u_top(x):
    if 0 < x < 1: return x
    if 1 <= x < 2: return 2 - x
    return 0.0  # esquinas

# === Entrada del usuario ===
tol_in = input("Ingresa la tolerancia TOL (ej. 1e-8): ").strip().replace(',', '.')
tol = 1e-8
try:
    if tol_in:
        tol = float(tol_in)
except Exception:
    print("No pude interpretar la tolerancia; usaré 1e-8.")
if tol <= 0:
    print("La tolerancia debe ser positiva; usaré 1e-8.")
    tol = 1e-8

nmax_in = input("Máximo de iteraciones N (enter para 1000): ").strip()
max_iter = 1000
try:
    if nmax_in:
        max_iter = int(nmax_in)
except Exception:
    print("No pude interpretar N; usaré 1000.")
if max_iter <= 0:
    print("N debe ser positivo; usaré 1000.")
    max_iter = 1000

# Ensamble del sistema y corrida de Jacobi
A, b, idx, Nx, Ny = arma_sistema_laplace_cuadrado(xs, ys, u_left, u_right, u_bottom, u_top)
x, k, tabla, ok = jacobi(A, b, tol=tol, max_iter=max_iter)

print(f"TOL usada: {tol}")
print("Iteraciones realizadas:", k)
print("¿Convergió?:", ok)
display(tabla)  # tabla ordenada por iteración

# Mapeo de solución al grid interior (j: y de abajo→arriba)
U_int = np.zeros((Ny, Nx))
for j in range(1, Ny+1):
    for i in range(1, Nx+1):
        U_int[j-1, i-1] = x[idx(i, j)]

df_grid = pd.DataFrame(U_int[::-1],
                      index=["y=4/3","y=2/3"],
                      columns=["x=2/3","x=4/3"])  # mostrar de arriba hacia abajo
print("\nValores aproximados en los puntos interiores (de arriba hacia abajo):")
display(df_grid)

print("\nValores de referencia (exactos en esta malla):")
print("u(2/3,2/3)=7/36,   u(4/3,2/3)=5/12,   u(2/3,4/3)=13/36,   u(4/3,4/3)=7/12")


Ingresa la tolerancia TOL (ej. 1e-8): 0.01
Máximo de iteraciones N (enter para 1000): 1000
TOL usada: 0.01
Iteraciones realizadas: 6
¿Convergió?: True


,k,x1,x2,x3,x4,error_inf
0,1,0.000000,0.222222,0.166667,0.388889,0.388889
1,2,0.097222,0.319444,0.263889,0.486111,0.097222
2,3,0.145833,0.368056,0.312500,0.534722,0.048611
3,4,0.170139,0.392361,0.336806,0.559028,0.024306
4,5,0.182292,0.404514,0.348958,0.571181,0.012153
5,6,0.188368,0.410590,0.355035,0.577257,0.006076



Valores aproximados en los puntos interiores (de arriba hacia abajo):


,x=2/3,x=4/3
y=4/3,0.355035,0.577257
y=2/3,0.188368,0.410590



Valores de referencia (exactos en esta malla):
u(2/3,2/3)=7/36,   u(4/3,2/3)=5/12,   u(2/3,4/3)=13/36,   u(4/3,4/3)=7/12
